<a href="https://colab.research.google.com/github/epvillanueva10/Datos/blob/main/BOA_Py6S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atmospheric correction of Sentinel 2 image using Py6S in Google Colab environment**

This is the first part of python codes used in the article. The codes are tested inside Google Colab environment using Hong Kong water as the study area.

Guidance and reference provided at the following websites are appreciated.

*   https://github.com/samsammurphy/gee-atmcorr-S2
*   https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md
*   https://blog.csdn.net/qq_45110581/article/details/108629636








# Step 1 - Set up Py6S in Google Colab

In [226]:
!gfortran -v
!wget http://rtwilson.com/downloads/6SV-1.1.tar
!tar xvf 6SV-1.1.tar
!cd 6SV1.1

Using built-in specs.
COLLECT_GCC=gfortran
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/11/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:amdgcn-amdhsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 11.4.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-11/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,m2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-11 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --enable-libphobos-checking=release --with-target-system-zlib=auto --enable-objc-gc=auto --enable-multiarch --disab

**Manual work required before executing the subsequent code**

Refer to comments below

In [227]:
# modify "makefile" from "FC = g77 $(FFLAGS)" to "FC = gfortran -std=legacy -ffixed-line-length-none -ffpe-summary=none $(FFLAGS)"
# upload modified "makefile" to /content/6SV1.1

import os
os.chdir("/content/6SV1.1")
!ls
!make
os.environ["PATH"]="/content/6SV1.1:"+os.environ["PATH"]
# test
!sixsV1.1 < /content/Examples/Example_In_1.txt
!pip install Py6S
from Py6S import *
SixS.test()

6SV1.1	     DISCRE.f	  LAKEW.o     MOCA5.f	      OSPOL.f	     PRINT_ERROR.o  TRUNCA.f
6SV-1.1.tar  DISCRE.o	  main.f      MOCA5.o	      OSPOL.o	     RAHMALBE.f     TRUNCA.o
AATSR.f      DUST.f	  main.o      MOCA6.f	      OXYG3.f	     RAHMALBE.o     US62.f
AATSR.o      DUST.o	  Makefile    MOCA6.o	      OXYG3.o	     RAHMBRDF.f     US62.o
ABSTRA.f     ENVIRO.f	  MAS.f       MODISALBE.f     OXYG4.f	     RAHMBRDF.o     VARSOL.f
ABSTRA.o     ENVIRO.o	  MAS.o       MODISALBE.o     OXYG4.o	     ROUJALBE.f     VARSOL.o
AEROPROF.f   EQUIVWL.f	  MERIS.f     MODISBRDF.f     OXYG5.f	     ROUJALBE.o     VEGETA.f
AEROPROF.o   EQUIVWL.o	  MERIS.o     MODISBRDF.o     OXYG5.o	     ROUJBRDF.f     VEGETA.o
AEROSO.f     ETM.f	  METEO.f     MODIS.f	      OXYG6.f	     ROUJBRDF.o     VERSALBE.f
AEROSO.o     ETM.o	  METEO.o     MODIS.o	      OXYG6.o	     SAND.f	    VERSALBE.o
AKTOOL.f     Examples	  METH1.f     MSS.f	      OZON1.f	     SAND.o	    VERSBRDF.f
AKTOOL.o     GAUSS.f	  METH1.o     MSS.o	      OZON1

0

# Step 2 - Define functions required in atmospheric correction

**Functions created by Sam Murphy**

Modified from https://github.com/samsammurphy/gee-atmcorr-S2

In [228]:
"""
atmospheric.py, Sam Murphy (2016-10-26)

Atmospheric water vapour, ozone and AOT from GEE

Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)

"""


import ee

class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

    # start of NEXT month
    m2 = m1.advance(1,'month')

    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()

    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.

    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)
    """

    # Point geometry required
    centroid = geom.centroid()

    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)

    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

    # water image
    water_img = ee.Image(water_ic.first())

    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)

    return water_Py6S_units



  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset

    OR

    uses our fill value (which is mean value for that latlon and day-of-year)

    """

    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):

      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

      # ozone image
      ozone_img = ee.Image(ozone_ic.first())

      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))

      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)

      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

      """

      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))

      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)

    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')

    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )

    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.

    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """

    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])


    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )

      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))

      return img


    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)

    return AOT

Import required libraries

In [229]:
import ee
from Py6S import *
from datetime import datetime
import math
import pandas as pd
import numpy as np
import os
import sys
import folium
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, timedelta

**Initialize Google Earth Engine session**

Need enter verification using GEE account

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0H_AhmODCUxdZpZq4UhoTUh5MM9LoxlyDawJsJ-wauk&tc=xfRJNc-5cWU40p1Lq56knQnEecJQITotvi71evCegHQ&cc=i3FE8FDA1lb5EVT_tXpkeIZMyOq8vs2lxpVmyszvXR4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXk1tkiloO_xMDhGefZ5rtlhXVEynBFB6f_AyOERVTK3FCqeGF1y7uQ

Successfully saved authorization token.


**Py6S function**

In [230]:
# Define Py6S function
# Modified from https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md

def func1(img):
  S2 = ee.Image(img)
  date = S2.date()
  # top of atmosphere reflectance
  toa = S2.divide(10000)

  info = S2.getInfo()['properties']
  scene_date = datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
  solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  aot = Atmospheric.aerosol(geom,date).getInfo()

  SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
  alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
  km = alt/1000 # i.e. Py6S uses units of kilometers

  # Instantiate
  s = SixS()

  # Atmospheric constituents
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Maritime # https://github.com/robintw/Py6S/blob/master/Py6S/Params/aeroprofile.py
  s.aot550 = aot

  # Earth-Sun-satellite geometry
  s.geometry = Geometry.User()
  s.geometry.view_z = 0               # always NADIR
  s.geometry.solar_z = solar_z        # solar zenith angle
  s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
  s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
  s.altitudes.set_sensor_satellite_level()
  s.altitudes.set_target_custom_altitude(km)

  def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    return Wavelength(bandSelect[bandname])

  def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    return rad

  def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref

  # all wavebands
  output = S2.select('QA60')
  for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
    print(band)
    output = output.addBands(surface_reflectance(band))

  return output


# Step 3 - Clouds and water mask

**Filter by date and aoi**

In [231]:
# Define the area of interest (AOI)
Clip = ee.FeatureCollection('projects/epvillanueva/assets/Clip')
AOI = Clip.geometry()
geom = AOI

# Define the dates
DATES = ['2022-11-24', '2023-01-27', '2023-02-03', '2023-03-17', '2023-03-27', '2023-04-11',
         '2023-04-26', '2023-05-16', '2023-06-05', '2023-07-05']

CLD_PRB_THRESH = 40  #	Cloud probability (%); values greater than are considered cloud
NIR_DRK_THRESH = 0.5 # Near-infrared reflectance; values less than are considered potential cloud shadow
CLD_PRJ_DIST = 2 # Maximum distance (km) to search for cloud shadows from cloud edges
BUFFER = 15 # Distance (m) to dilate the edge of cloud-identified objects
CLOUD_FILTER = 30

**Non-water and clouds masks**

In [232]:
# Funtion to add no-water mask
def add_not_water_mask(img):
    # NDWI
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    # mNDWI
    mndwi = img.normalizedDifference(['B3', 'B11']).rename('mNDWI')

    mndwi_water = mndwi.gt(-0.05)
    ndwi_water = ndwi.gt(0.3)

    combined_water = mndwi_water.Or(ndwi_water).rename('COMBINED_WATER')
    not_water = combined_water.Not().rename('NOT_WATER')
    return img.addBands(not_water)

# Funtion to add cloud mask
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value (1 clouds)
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

# Funtion to add cloud shadow mask
def add_shadow_bands(img):
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 10})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Combined mask
def add_cld_shdw_mask(img):

    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/10)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 10})
        .rename('cloudmask'))

    # Añadir la Máscara Final a la Imagen:
    return img_cloud_shadow.addBands(is_cld_shdw)

**Import colecction**

In [233]:
# Function to apply corrections and add mask
def apply_corrections(image):
    corrected_img = func1(image)
    masked_img = add_cld_shdw_mask(add_not_water_mask(corrected_img))
    return masked_img

# Import and filter S2 and s2cloudless and add mask
def get_s2_sr_cld_col(aoi, dates, search_window=10):
    def process_date(date_str):
        target_date = datetime.strptime(date_str, '%Y-%m-%d')
        start_search = target_date - timedelta(days=search_window)
        end_search = target_date + timedelta(days=search_window)

        s2_sr_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
                     .filterBounds(aoi)\
                     .filterDate(start_search.strftime('%Y-%m-%d'), end_search.strftime('%Y-%m-%d'))\
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))

        s2_cloudless_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
                            .filterBounds(aoi)\
                            .filterDate(start_search.strftime('%Y-%m-%d'), end_search.strftime('%Y-%m-%d'))

        joined_col = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        }))

        def date_diff(image):
            image_date = ee.Date(image.get('system:time_start'))
            diff = ee.Number(target_date.timestamp() * 1000).subtract(image_date.millis()).abs()
            return image.set('dateDiff', diff)

        nearest_image = joined_col.map(date_diff).sort('dateDiff').first()
         # Retornar la imagen y la fecha seleccionada
        selected_date = ee.Date(nearest_image.get('system:time_start')).format('YYYY-MM-dd')
        cloud_percentage = nearest_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
        print(f'Fecha original: {date_str}, Fecha seleccionada: {selected_date.getInfo()} Porcentaje de nubosidad: {cloud_percentage}%')

        return apply_corrections(nearest_image.clip(aoi))

    return [process_date(date) for date in dates]

In [234]:
# Apply functions to collections
collections = get_s2_sr_cld_col(AOI, DATES)

Fecha original: 2022-11-24, Fecha seleccionada: 2022-11-27 Porcentaje de nubosidad: 23.7657159917091%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-01-27, Fecha seleccionada: 2023-01-26 Porcentaje de nubosidad: 0.307621597213263%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-02-03, Fecha seleccionada: 2023-01-31 Porcentaje de nubosidad: 0.519043245665088%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-03-17, Fecha seleccionada: 2023-03-17 Porcentaje de nubosidad: 13.6425670580218%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-03-27, Fecha seleccionada: 2023-03-27 Porcentaje de nubosidad: 0.0173312245320949%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-04-11, Fecha seleccionada: 2023-04-06 Porcentaje de nubosidad: 6.17959419938198%
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
Fecha original: 2023-04-26, Fecha seleccionada: 2023-04-26 Porcentaje de nubosidad: 10.3341075851036%
B1
B2
B3
B4
B5
B6
B7
B8


**Visualization**

In [235]:
# Función para añadir capas de imágenes a Folium
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
    ).add_to(self)

# Agregar la función add_ee_layer a la clase Map de Folium
folium.Map.add_ee_layer = add_ee_layer

# Función para generar y agregar las capas de una fecha específica al mapa
def generate_layers_for_date(img, map_object):
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability').selfMask()
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform').selfMask()
    not_water = img.select('NOT_WATER').selfMask()

    # Agregar capas al mapa
    map_object.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.25, 'gamma': 1.1}, 'S2 image')
    # map_object.add_ee_layer(probability, {'min': 0, 'max': 100}, 'Probability (cloud)')
    # map_object.add_ee_layer(clouds, {'palette': 'e056fd'}, 'Clouds')
    # map_object.add_ee_layer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'Cloud Transform')
    # map_object.add_ee_layer(dark_pixels, {'palette': 'orange'}, 'Dark Pixels')
    # map_object.add_ee_layer(shadows, {'palette': 'yellow'}, 'Shadows')
    map_object.add_ee_layer(cloudmask, {'palette': 'orange'}, 'Cloud Mask')
    map_object.add_ee_layer(not_water, {'palette': 'blue'}, 'Not Water')

# Función para visualizar las capas con un selector de fechas
def display_cloud_layers_with_date_selector(collections, dates, aoi):
    def update_map(date_index):
        # Crear un nuevo objeto de mapa para la fecha seleccionada
        center = aoi.centroid(10).coordinates().reverse().getInfo()
        m = folium.Map(location=center, zoom_start=15)

        img = collections[date_index]
        generate_layers_for_date(img, m)
        m.add_child(folium.LayerControl())

        # Mostrar el mapa actualizado
        display(m)

    # Crear el selector de fechas
    date_selector = widgets.Dropdown(
        options=[(date, i) for i, date in enumerate(dates)],
        value=0,
        description='Select Date:',
    )

    # Vincular el selector de fechas con la función de actualización del mapa
    widgets.interact(update_map, date_index=date_selector)

In [236]:
display_cloud_layers_with_date_selector(collections, DATES, AOI)

interactive(children=(Dropdown(description='Select Date:', options=(('2022-11-24', 0), ('2023-01-27', 1), ('20…

**Apply mask**

In [237]:
def apply_masks_and_clip(img):
    # Aplicar máscaras de no-agua, nubes y sombras
    masked_img = add_cld_shdw_mask(add_not_water_mask(img))

    # Invertir las máscaras para mantener solo las áreas no enmascaradas ( 0 = sin nubes, sin sombras, y con agua)
    cloud_shadow_mask = masked_img.select('cloudmask').Not()
    water_mask = masked_img.select('NOT_WATER').Not()

    # Combinar las máscaras para mantener solo las áreas no enmascaradas (no nubes, no sombras, y agua)
    final_mask = cloud_shadow_mask.And(water_mask)
    return masked_img.updateMask(final_mask).clip(AOI)

def visualize_processed_images(processed_collections, dates, aoi):
    def update_map(date_index):
        # Crear un nuevo objeto de mapa para la fecha seleccionada
        center = aoi.centroid(10).coordinates().reverse().getInfo()
        m = folium.Map(location=center, zoom_start=12)

        img = processed_collections[date_index]
        # Aquí solo se visualiza la imagen recortada y procesada
        m.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.25, 'gamma': 1.1}, 'Processed S2 image')
        m.add_child(folium.LayerControl())

        # Mostrar el mapa actualizado
        display(m)

    # Crear el selector de fechas
    date_selector = widgets.Dropdown(
        options=[(date, i) for i, date in enumerate(dates)],
        value=0,
        description='Select Date:',
    )

    # Vincular el selector de fechas con la función de actualización del mapa
    widgets.interact(update_map, date_index=date_selector)

# Procesar las imágenes con las máscaras y recorte
processed_collections = [apply_masks_and_clip(img) for img in collections]

# Visualizar las imágenes procesadas
visualize_processed_images(processed_collections, DATES, AOI)

interactive(children=(Dropdown(description='Select Date:', options=(('2022-11-24', 0), ('2023-01-27', 1), ('20…

# Extract satellite date

In [238]:
stations = ee.FeatureCollection('projects/epvillanueva/assets/Estaciones')

# Función para extraer datos de las imagenes en las estaciones
def extract_data_for_stations(processed_collections, stations):
    all_results = []

    # Obtener la lista de IDs de las estaciones
    station_ids = stations.aggregate_array('system:index').getInfo()

    for img in processed_collections:
        # Extraer la fecha de la imagen
        image_date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

        for station_id in station_ids:
            # Obtener la estación específica por ID
            station = stations.filter(ee.Filter.eq('system:index', station_id)).first()

            # Coordenadas y nombre de la estación
            coords = station.geometry().coordinates().getInfo()
            station_name = station.get('Estacion').getInfo()

            # Extraer los valores de la imagen en las coordenadas de la estación
            values = img.reduceRegion(
                ee.Reducer.toList(),
                ee.Geometry.Point(coords),
                10
            ).getInfo()

            # Convertir listas vacías a NaN y preparar el diccionario de resultados
            result = {
                'station': station_name,
                'date': image_date,
                'coordinates': coords
            }
            for band in values.keys():
                result[band] = values[band][0] if values[band] else float('nan')

            all_results.append(result)

    return all_results

# Aplicar función a colección completa
station_data = extract_data_for_stations(processed_collections, stations)

In [239]:
# Convertir a DataFrame
df = pd.DataFrame(station_data)

# Mostrar el DataFrame
df.head

<bound method NDFrame.head of      station        date                               coordinates        B1  \
0         16  2022-11-27  [-74.84870447912535, 11.046055986375878]       NaN   
1         19  2022-11-27   [-74.85352030826307, 11.05052872452565]       NaN   
2          2  2022-11-27  [-74.85720798483428, 11.036825824995978]       NaN   
3         24  2022-11-27  [-74.84229675091156, 11.056366814388284]       NaN   
4         18  2022-11-27  [-74.83878297928143, 11.046213539954396]       NaN   
..       ...         ...                                       ...       ...   
295       14  2023-07-10  [-74.85791252280072, 11.046156645616358]  0.098529   
296       29  2023-07-10  [-74.85399297297474, 11.064226607844894]  0.084216   
297       28  2023-07-10  [-74.84897202518856, 11.059815346597949]  0.083946   
298       30  2023-07-10  [-74.84898986159277, 11.064226607844894]  0.087861   
299       27  2023-07-10  [-74.85403310488422, 11.059740949757611]  0.086106   

         

In [240]:
df.columns

Index(['station', 'date', 'coordinates', 'B1', 'B10', 'B11', 'B12', 'B2', 'B3',
       'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'NOT_WATER', 'NOT_WATER_1',
       'QA60', 'cloud_transform', 'cloud_transform_1', 'cloudmask',
       'cloudmask_1', 'clouds', 'clouds_1', 'dark_pixels', 'dark_pixels_1',
       'probability', 'probability_1', 'shadows', 'shadows_1'],
      dtype='object')

Bandas del Sentinel-2

- B1 - Banda Costera (Coastal Aerosol)

Longitud de Onda: 433 - 453 nm
Uso: Diseñada principalmente para el monitoreo costero y atmosférico. Útil para estudiar el contenido de sedimentos y partículas en zonas costeras y cuerpos de agua.
- B2 - Banda Azul (Blue)

Longitud de Onda: 458 - 523 nm
Uso: Útil para estudios de cuerpos de agua (en especial para determinar la turbidez) y como una de las bandas fundamentales para la generación de imágenes en color natural.
- B3 - Banda Verde (Green)

Longitud de Onda: 543 - 578 nm
Uso: Importante para la evaluación de la vegetación y la determinación de índices como NDVI. También se usa en la generación de imágenes en color natural.
- B4 - Banda Roja (Red)

Longitud de Onda: 650 - 680 nm
Uso: Fundamental para la evaluación de la vegetación, índices de vegetación y estudios sobre la salud de las plantas.
- B5 - Banda del Borde Rojo (Red Edge)

Longitud de Onda: 698 - 713 nm
Uso: Útil para detectar cambios en la clorofila y en el contenido de nitrógeno de la vegetación.
- B6 - Banda del Borde Rojo (Red Edge)

Longitud de Onda: 733 - 748 nm
Uso: Similar a B5, ayuda en la evaluación del estado de la vegetación y es útil en la agricultura de precisión.
- B7 - Banda del Borde Rojo (Red Edge)

Longitud de Onda: 773 - 793 nm
Uso: Proporciona información adicional sobre la vegetación, especialmente útil para determinar la biomasa y la salud de la vegetación.
- B8 - Banda del Infrarrojo Cercano (NIR)

Longitud de Onda: 785 - 900 nm
Uso: Esencial para la evaluación de la vegetación, índices de vegetación como NDVI, y para estudios de hidrología y uso del suelo.
- B8A - Banda del Infrarrojo Cercano (NIR)

Longitud de Onda: 855 - 875 nm
Uso: Similar a B8 pero con un enfoque más específico en la evaluación detallada de la vegetación.
- B9 - Banda del Vapor de Agua

Longitud de Onda: 935 - 955 nm
Uso: Útil para la medición de humedad y vapor de agua en la atmósfera.
- B10 - Banda SWIR (Short-Wave Infrared)

Longitud de Onda: 1365 - 1385 nm
Uso: Principalmente para aplicaciones atmosféricas, como la detección de vapor de agua y cirros.
- B11 - Banda SWIR

Longitud de Onda: 1565 - 1655 nm
Uso: Utilizada para aplicaciones agrícolas y geológicas, y para la discriminación de nieve/hielo y nubes.
- B12 - Banda SWIR

Longitud de Onda: 2100 - 2280 nm
Uso: Útil para la discriminación de tipos de vegetación y la detección de humedad en suelos y vegetación.

- QA60: Es una banda de control de calidad que proporciona información sobre nubes, condiciones atmosféricas y otros aspectos de calidad de la imagen.

**Con base en esto, se eliminarán todas las mascaras y las bandas 9 y 10 correspondientes a información atmosferica y no de superficie**

In [243]:
# Organizar datos satelitales
columnas_deseadas = ['station', 'date', 'coordinates', 'B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A']
datos_satelitales = df[columnas_deseadas]
datos_satelitales = datos_satelitales.sort_values(by=['date', 'station'])
datos_satelitales.replace('', np.nan, inplace=True)
datos_satelitales['date'] = pd.to_datetime(datos_satelitales['date'])
datos_satelitales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 5 to 298
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   station      300 non-null    int64         
 1   date         300 non-null    datetime64[ns]
 2   coordinates  300 non-null    object        
 3   B1           206 non-null    float64       
 4   B11          206 non-null    float64       
 5   B12          206 non-null    float64       
 6   B2           206 non-null    float64       
 7   B3           206 non-null    float64       
 8   B4           206 non-null    float64       
 9   B5           206 non-null    float64       
 10  B6           206 non-null    float64       
 11  B7           206 non-null    float64       
 12  B8           206 non-null    float64       
 13  B8A          206 non-null    float64       
dtypes: datetime64[ns](1), float64(11), int64(1), object(1)
memory usage: 35.2+ KB


In [245]:
datos_satelitales.to_csv('datos_satelitales_msk.csv', index=False)